In [34]:
import itertools

words = open('../data/names.txt', 'r').read().splitlines()
words[:10]

# words = ["giorgio", "gina"]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [7]:
import torch

In [93]:
chars = sorted(list(set(''.join(words))))

In [383]:
inputs = list(itertools.product(chars, repeat=2))
inputs += [('.', c) for c in chars]
inputs[:10]

[('a', 'a'),
 ('a', 'b'),
 ('a', 'c'),
 ('a', 'd'),
 ('a', 'e'),
 ('a', 'f'),
 ('a', 'g'),
 ('a', 'h'),
 ('a', 'i'),
 ('a', 'j')]

In [384]:
stoi = {s:i for i, s in enumerate(inputs)}
itos = {i:s for s, i in stoi.items()}
stoo = {s:i+1 for i, s in enumerate(chars)}
stoo['.'] = 0
otos = {i:s for s, i in stoo.items()}
n_tokens = len(stoi)
n_tokens

702

In [385]:
N = torch.zeros((n_tokens, 27), dtype=torch.int32)

In [386]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for i in range(len(chs)-2):
        ixi = stoi[tuple(chs[i:i+2])]
        ixo = stoo[chs[i+2]]
        N[ixi, ixo] += 1

In [387]:
P = N.float()
P /= P.sum(1, keepdims=True)

In [222]:
P[676]

tensor([0.0000, 0.0469, 0.0431, 0.0070, 0.0830, 0.0125, 0.0048, 0.0039, 0.0206,
        0.0349, 0.0061, 0.0170, 0.1433, 0.0871, 0.1413, 0.0023, 0.0039, 0.0020,
        0.1093, 0.0440, 0.0163, 0.0345, 0.0551, 0.0014, 0.0061, 0.0392, 0.0345])

In [388]:
# g = torch.Generator().manual_seed(2147483647)
start_letter = 'a'
for i in range(5):
    out = [start_letter]
    ix = stoi[('.', start_letter)]
    while True:
        p = P[ix]
        ixo = torch.multinomial(p, num_samples=1, replacement=True).item()
        out.append(otos[ixo])
        # print(itos[ix], ix)
        # print(otos[ixo], ixo)
        if otos[ixo] == '.':
            break
        ix = stoi[itos[ix][1], otos[ixo]]

    print(''.join(out))

alashdiyassann.
annleah.
agday.
asiabduliah.
ah.


In [22]:
# GOAL: to maximise the likelihood of the data w.r.t. the model parameters
# (statistical modelling). what mean this ? we can't realistically get 100%
# probability. let's say:
# our training set is ['giorgio', 'gina']:
# there is no way to have 100% probability for what follows 'i',
# since it could either be bigram 'io' or 'in'. But we want to maximise the
# chance of them showing up, and correctly: as close to 1/3 for 'in' and 2/3
# for 'io' as possible. If we make 'io' much higher, the likelihood of the
# rest of the data ('g<in>a') showing up becomes much lower, and so the
# overall quality of the model is lower. MAXIMISE THE LIKELIHOOD OF THE
# DATA WITH RESPECT TO THE MODEL PARAMETERS. CAN'T HAVE ALL 1s AS
# PROBABILITIES SINCE THEY WILL CLASH - JUST LOOKING FOR WHAT'S BEST. WITH
# BIGRAMS DATA, THE CALCULATED LOSS W.R.T TRAINING DATA IS AS GOOD AS IT'S
# GOING TO GET.
# so we want to maximise the product of all probabilities.
# i.e. maximise the log likelihood (since probs are 0-1, the logs are gonna
# be negative, and we want to get close to 0)
# i.e. minimise negative log likelihood (positive number == loss, want it to
# be as low as possible)
# i.e. minimise average negative log likelihood.

# The model is good when the probabilities for each bigram are high. Why?
# See above.


In [390]:
log_likelihood = 0.0
n = 0
for w in words:
    chs = ['.'] + list(w) + ['.']
    for i in range(len(w)-2):
        ix1, ix2 = stoi[tuple(w[i:i+2])], stoo[w[i+2]]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        # print(f'{ch1}{ch2}: {prob:.4f} {logprob:.4f}')

print(f'{log_likelihood=}')
nll = -log_likelihood

print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-303102.5938)
nll=tensor(303102.5938)
2.295414447784424


In [338]:
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.nn.functional import cross_entropy, nll_loss, log_softmax

In [248]:
# create the training set of all the trigrams
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for i in range(len(chs)-2):
        ix1 = stoi[''.join(chs[i:i+2])]
        ix2 = stoo[chs[i+2]]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples:', num)

number of examples: 196113


In [285]:
L = 100
R = 0.01

In [312]:
def forward_pass(inputs, weights):
    # xenc = F.one_hot(inputs, num_classes=n_tokens).float() # input to the NN: one-hot enc
    # logits = xenc @ weights # predict log-counts
    logits = weights[inputs]
    counts = logits.exp() # convert to counts (analogous to N above)
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next char

    return probs

In [341]:
def mean_nll(probs, n_inputs, labels):
    return -probs[torch.arange(n_inputs), labels].log().mean() # + R*(W**2).mean()

In [322]:
train_ixs, dev_ixs, test_ixs = random_split(range(num), [0.8, 0.1, 0.1])
train_xs, train_ys = xs[train_ixs], ys[train_ixs]
dev_xs, dev_ys = xs[dev_ixs], ys[dev_ixs]
test_xs, test_ys = xs[test_ixs], ys[test_ixs]

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((n_tokens, 27), generator=g, requires_grad=True)

In [360]:
# gradient descent
for k in range(100):
    probs = forward_pass(xs, W)

    # print(f'{W.shape=}')
    # print(f'{xenc.shape=}')
    # print(f'{probs.shape=}')
    # print(f'{ys.shape=}')
    loss = mean_nll(probs, len(xs), ys)
    # second arg normalises to bring Ws to 0. equivalent to (N+1) from above.
    # unsure why we do this here though.
    print(f'{loss:=.4f}')

    # second-last 2 lines are softmax = go from logits (some positive & negative
    # numbers) to a probability distribution.

    # backward pass
    W.grad = None  # set gradient to 0
    loss.backward()

    # update
    W.data += -L * W.grad

2.0842
2.0842
2.0842
2.0842
2.0842
2.0842
2.0842
2.0841
2.0841
2.0841
2.0841
2.0841
2.0841
2.0841
2.0840
2.0840
2.0840
2.0840
2.0840
2.0840
2.0840
2.0839
2.0839
2.0839
2.0839
2.0839
2.0839
2.0839
2.0838
2.0838
2.0838
2.0838
2.0838
2.0838
2.0838
2.0837
2.0837
2.0837
2.0837
2.0837
2.0837
2.0837
2.0836
2.0836
2.0836
2.0836
2.0836
2.0836
2.0836
2.0835
2.0835
2.0835
2.0835
2.0835
2.0835
2.0835
2.0834
2.0834
2.0834
2.0834
2.0834
2.0834
2.0834
2.0834
2.0833
2.0833
2.0833
2.0833
2.0833
2.0833
2.0833
2.0832
2.0832
2.0832
2.0832
2.0832
2.0832
2.0832
2.0831
2.0831
2.0831
2.0831
2.0831
2.0831
2.0831
2.0831
2.0830
2.0830
2.0830
2.0830
2.0830
2.0830
2.0830
2.0829
2.0829
2.0829
2.0829
2.0829
2.0829
2.0829


In [324]:
dev_loss = mean_nll(forward_pass(dev_xs, W), len(dev_xs), dev_ys)
print(f'{dev_loss.item()=:.4f}')
test_loss = mean_nll(forward_pass(test_xs, W), len(test_xs), test_ys)
print(f'{test_loss.item()=:.4f}')

dev_loss.item()=3.6502
test_loss.item()=3.6595


In [ ]:
# Softmax & Sigmoid are output layers. Softmax returns a probability distribution - increasing the
# likelihood of one class reduces the likelihood of others (= multiclass classification. Only 1 class).
# Sigmoid gives 0-1 per each class. They could sum to greater than 1. (= multilabel classification.
# multiple classes could be valid).

# You can mix and match output layers with cost functions.

# Cross-entropy and NLL (negative log-likelihood, similar to maximum likelihood estimation) are cost
# functions. In multiclass classification, they are equivalent, because cross-entropy over a softmax
# output layer (i.e. a probability distribution over all examples) is the same as NLL. In multi-label
# classification, you can't have a prob. distribution over all examples (like the NLL equation), so
# there is no equivalence. You can do binary cross-entropy here, interpreting each output neuron as
# its own probability p (YES) and 1-p (NO), then sum over all these probabilities.

In [ ]:
# Cross-entropy and NLL both reward network for high probabilities in correct output classes,
# but Cross-entropy also penalises network for high probabilities in the wrong classes. NLL alone
# doesn't do this, but we achieve something similar using the regularisation factor from above
# (which pushes all weights to 0).

In [28]:
# NN approach is converging on the final loss of the table-probability approach
# because the table approach is as good as we can get it. Essentially, it is
# perfect. The problem arises when we expand from bigrams. Tri-grams, 4-grams,
# the tables for these become unwieldy (4-gram: need all 3-char combination
# inputs leading to an output). It is un-scalable.
# NNs (and the gradient descent approach) are scalable - we just change the
# way to get the logits (the forward pass), and everything else stays the same.
# Stick a softmax to the end of it, gradient descent, adjust weights. Easy.

In [29]:
# WOW OK
# The W we end up with is the table N we calculated by counting from above!!
# The comparison: Matrix multiplying a one-hot encoded vector by a matrix
# is equivalent to 'selecting' the row represented by that vector (if we
# had 5 as the one-hot, we select row 5 of W). Which is what we do with N:
# based on the first character, pick that row in the table, and you have a row
# of counts for each character that could follow it.
# Damnnnnnnnnn.
# Well, W is the log-counts. So W.exp() kinda equals N. Difference is, we
# counted everything to get to N, while we started with random values and
# grad-descented to what W is to get the right logcounts/counts/probabilities.